In [1]:
'''
CursorFlowASTFactory를 통해 FlowAST 를 생성해야한다
이를 위해 Cursor 및 FlowAST 를 태스트 하기 위한 노트
'''
import sys
sys.path.append(r'D:\dev\Python\clangParserProjects\MakeStateNode')
sys.path.append(r'D:\dev\python_pure_projects\PyUtil')

In [2]:
from neo4j_manager.neo4jHandler import Neo4jHandler
db = Neo4jHandler(uri ="bolt://localhost:7687", user="neo4j", 
                  password="123456789", database = "flowast")
db.print_info()
db.delete_all_nodes()



Nodes
Relationships


In [3]:
from clangParser.datas.CUnit import CUnit, Cursor
unit=CUnit.parse(r'D:\dev\AutoPlanning\trunk\AP_trunk_Task\mod_APImplantSimulation\ActuatorHybridFixture.cpp')

from oms.dataset.flowAST.factory_flowAST import CursorVisitor, FlowAST

libclang.dll 설정 실패
library file must be set before before using any other functionalities in libclang.


In [4]:
#유닛의 메서드를 순환하여 visitor 와 대응
from oms.dataset.flowAST.factory_flowAST import CursorVisitor, FlowAST
import clangParser.clang_utill as ClangUtill

skip_list = ['VAR_DECL', 'UNEXPOSED_DECL', 'USING_DIRECTIVE']
visitor_map = {}

for method_cursor in unit.get_this_Cursor():
    if method_cursor.kind in skip_list:
        continue
    visitor_map[method_cursor] = CursorVisitor(method_cursor)

print(len(visitor_map))

97


In [5]:
#하나의 메서드에 대해서 db에 저장 처리
from oms.dataset.flowAST.flowAST import IfFlowInfo, WhileFlowInfo, FlowInfo
def check_save_data(save_datas: list[FlowAST], save_relations:list[tuple[FlowAST, FlowAST, str]], visitor: CursorVisitor):
    print(len(save_datas), len(save_relations), '->', len(set(save_relations)))
    if len(save_datas) != len(visitor.cursor2node):
        print(f'cursor2node 개수가 다름 map : save = {len(visitor.cursor2node)} != {len(save_datas)}')
        for k, v in visitor.cursor2node.items():
            if v not in save_datas:
                print(k.get_src_name(), ' not saved')
        print('======================')

    # #시작과 종료 노드인 2개 curor2db에 없음음
    # for node in save_datas:
    #     assert node in visitor.cursor2node.values(), f'save_datas 에 없는 노드 {node.kind}'
    for node, next_node, rel_type in save_relations:
        assert node is not None and next_node is not None, 'None 노드 존재 ' + rel_type



def save_to_db(visitor: CursorVisitor, db: Neo4jHandler = db):
    save_datas = set()
    save_relations = []

    def visit(node : FlowAST):
        if node is None or node in save_datas:
            return
        else:
            save_datas.add(node)
            if node.next_flow:
                save_relations.append( (node, node.next_flow, 'next'))
                visit(node.next_flow)

            extented_info = node.extended_info
            if extented_info:
                if isinstance(extented_info, IfFlowInfo) or isinstance(extented_info, WhileFlowInfo):
                    cond = extented_info.condition
                    assert cond, 'If While 에 조건 식 없음'
                    save_relations.append( (node, cond, 'condition'))
                    visit(cond)
                    
                    save_relations.append( (node, extented_info.else_flow, 'else'))
                    visit(extented_info.else_flow)

                    
    visit(visitor.head_node)

    check_save_data(save_datas, save_relations, visitor)


    # assert len(visitor.cursor2node) == len(save_datas), 'cursor2node 개수가 다름\n' \
    # + f'{len(visitor.cursor2node)} != {len(save_datas)}'

    save_datas.add(visitor.source_cursor)
    save_relations.append( (visitor.source_cursor, visitor.head_node, 'head_node'))

    print(f'\r{visitor.source_cursor.get_src_name()}', end='')
    db.save_data(list(save_datas))
    db.add_relationship(save_relations)

db.delete_all_nodes()
for visitor in visitor_map.values():
    save_to_db(visitor)




34 33 -> 33
cursor2node 개수가 다름 map : save = 32 != 34
CActuatorHybridFixture.CActuatorHybridFixture()62 61 -> 61
cursor2node 개수가 다름 map : save = 52 != 62
CActuatorHybridFixture.~CActuatorHybridFixture()61 60 -> 60
cursor2node 개수가 다름 map : save = 58 != 61
CActuatorHybridFixture.OnAttach()13 12 -> 12
cursor2node 개수가 다름 map : save = 10 != 13
CActuatorHybridFixture._setAlphaBlend(CImage*)68 67 -> 67
cursor2node 개수가 다름 map : save = 63 != 68
CActuatorHybridFixture.OnInitialUpdate()7 6 -> 6
cursor2node 개수가 다름 map : save = 5 != 7
CActuatorHybridFixture.InitializeData()14 13 -> 13
cursor2node 개수가 다름 map : save = 12 != 14
CActuatorHybridFixture.GetDistanceViewCenter()45 46 -> 46
cursor2node 개수가 다름 map : save = 41 != 45
CActuatorHybridFixture.renderViewPlane()14 13 -> 13
cursor2node 개수가 다름 map : save = 12 != 14
CActuatorHybridFixture.Fixture_Move(UINT)10 9 -> 9
cursor2node 개수가 다름 map : save = 7 != 10
CActuatorHybridFixture.Fixtrue_Up(UINT)5 4 -> 4
cursor2node 개수가 다름 map : save = 3 != 5
CActuatorHy